## Prerequisits

The notebook requires.
- ECU simulation framework wheel.
- _RemotiveBroker_ accessible with a with interface name `vcan0` (can be implemented using UDP)

URL pointing to the broker:

In [ ]:
import os

url = os.environ["REMOTIVE_BROKER_URL"] if "REMOTIVE_BROKER_URL" in os.environ else "http://remotive-broker:50051"

## Presentation

### Import python libraries

In [ ]:
import asyncio

from remotivelabs.broker import BrokerClient, Frame, RestbusFrameConfig, RestbusSignalConfig, WriteSignal
from remotivelabs.topology.behavioral_model import BehavioralModel
from remotivelabs.topology.namespaces import filters
from remotivelabs.topology.namespaces.can import CanNamespace, RestbusConfig

### Read data

Just read a signal and print the value.

In [ ]:
async def _run():
    async with BrokerClient(url=url) as c:
        signal_batches = await c.subscribe(("vcan0", ["MyFrame.MySignal"]))
        async for batch in signal_batches:
            print(batch)


task = asyncio.create_task(_run())

In [ ]:
task.cancel()

* Send data using udptools or cansend!

### ECU 1, Simple ECU simulation

Echos out a counter on the CAN bus. "Rest bus".

In [ ]:
class MyEcuBehaviour:
    can_bus: CanNamespace

    def __init__(self, can_bus: CanNamespace):
        self.total_sum = 0
        self.can_bus = can_bus

    async def on_my_frame(self, frame: Frame):
        self.total_sum += int(frame.signals["MyFrame.MySignal"])
        print(f"Adding value {frame.signals['MyFrame.MySignal']}. Echoing out sum: {self.total_sum}.")
        await self.can_bus.restbus.update_signals(RestbusSignalConfig.set(name="MySum.MyCount", value=self.total_sum))


c = BrokerClient(url=url)
can_bus = CanNamespace(
    "vcan0",
    c,
    restbus_configs=[RestbusConfig([filters.FrameFilter(frame_name="MySum")])],
)
my = MyEcuBehaviour(can_bus)

stub = BehavioralModel(
    "ECU1",
    namespaces=[can_bus],
    broker_client=c,
    input_handlers=[can_bus.create_input_handler([filters.FrameFilter("MyFrame")], my.on_my_frame)],
)


async def _run_stub(s: BehavioralModel):
    async with c, s:
        await s.run_forever()


task = asyncio.create_task(_run_stub(stub))

In [ ]:
task.cancel()

* Send data using udptools or cansend!

### ECU 2, Integration with GUI

Example integration with _IPyWidgets_.

In [ ]:
import ipywidgets as ipw

%gui asyncio

In [ ]:
ui_float1 = ipw.FloatProgress(min=0, max=20.0, description="Signal value")
ui_slider1 = ipw.IntSlider(min=0, max=512, description="Value to send", style=dict(description_width="initial"))
ui_button1 = ipw.Button(description="Send a message", button_style="info")
ipw.VBox(
    [
        ipw.HTML(value="<b>My ECU</b>"),
        ui_float1,
        ui_slider1,
        ui_button1,
    ]
)

In [ ]:
async def on_my_frame(frame: Frame):
    ui_float1.value = frame.signals["MyFrame.MySignal"]


c = BrokerClient(url=url)
can_bus = CanNamespace("vcan0", c)


async def on_button_click():
    await can_bus.publish(WriteSignal(name="MySum.MyCount", value=ui_slider1.value))


stub = BehavioralModel(
    "ECU2",
    namespaces=[can_bus],
    broker_client=c,
    input_handlers=[can_bus.create_input_handler([filters.FrameFilter("MyFrame")], on_my_frame)],
)


async def _run_stub(s: BehavioralModel):
    async with c, s:
        await s.run_forever()


task = asyncio.create_task(_run_stub(stub))
ui_button1.on_click(lambda _: asyncio.get_event_loop().create_task(on_button_click()))

In [ ]:
task.cancel()

* Try click "Send a message" button. 
* Send data using udptools or cansend!

### ECU 3, Restbus GUI

Run a restbus and control the output with a GUI.

In [ ]:
import ipywidgets as ipw

%gui asyncio

In [ ]:
c = BrokerClient(url=url)

can_bus = CanNamespace("vcan0", c, restbus_configs=[RestbusConfig([filters.FrameFilter(frame_name="MySum")], delay_multiplier=0.1)])
await can_bus.open()
await can_bus.restbus.start()


def ui_restbus_output(x: int):
    async def _update():
        await can_bus.restbus.update_signals(RestbusSignalConfig.set(name="MySum.MyCount", value=x))  # Alt1. Update restbus output
        # await can_bus.publish(WriteSignal("MySum.MyCount", x)) # Alt2. Sent value once

    asyncio.get_event_loop().create_task(_update())


ipw.interact(ui_restbus_output, x=ipw.IntSlider(value=0xFF, min=0, max=0xFFFF))

In [ ]:
await can_bus.close()
await c.disconnect()

Drag the slider to update the value being sent out on the bus.

See comment `Alt1` which covers updating the value sent on the restbus. See comment `Alt2` for how to do a direct publish instead of updating the periodic value sent out on the rest bus. Both `Alt1` and `Alt2` can be called in the same callback.

### ECU 4, two stage ECU

1. Boot. Until MyFrame is received
2. Drive mode. Add frame to restbus

In [ ]:
phase_boot_completed = asyncio.Future()


async def on_my_frame(_frame: Frame):
    print("on_my_frame")
    phase_boot_completed.set_result(None)


c = BrokerClient(url=url)
can_bus = CanNamespace("vcan0", c)

stub = BehavioralModel(
    "ECU3",
    namespaces=[can_bus],
    broker_client=c,
    input_handlers=[can_bus.create_input_handler([filters.FrameFilter("MyFrame")], on_my_frame)],
)


async def _run_stub(s: BehavioralModel):
    async with c, s:
        await phase_boot_completed
        await can_bus.restbus.add(RestbusFrameConfig(name="MySum", cycle_time=1000), start=True)
        await s.run_forever()


task = asyncio.create_task(_run_stub(stub))

In [ ]:
task.cancel()